# Загрузка фотографий всех строющихся фотографий с сайта "наш.дом.рф"

In [1]:
import requests
import pandas as pd

In [2]:
# используем ранее полученные данные
data = pd.read_pickle('houses_dataset')
data.head()

,id,pdId,region,address,nameObj,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,...,developer.bankruptStage.bankruptStageDesc,developer.bankruptStage.bankruptLawUrl,developer.orgBankruptMsgDttm,metro.id,metro.name,metro.line,metro.color,metro.time,metro.isWalk,metro.colors
0,1284,1527,77,"г Москва, район внутригородское муниципальное ...","Жилой Комплекс ""Мир Митино"" (сокращенное наиме...",27.0,27.0,1115,2024-12-31,Блоки,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1366,1474,23,"г Краснодар, улица им. Героя Яцкова И.В., лите...","Микрорайон ""Губернский""",25.0,25.0,207,2025-06-30,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1367,1474,23,"г Краснодар, улица им. Героя Яцкова И.В., лите...","Микрорайон ""Губернский""",1.0,1.0,0,2025-06-30,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1368,1476,23,"г Краснодар, улица им. Героя Яцкова И.В., лите...","Микрорайон ""Губернский""",1.0,19.0,321,2025-06-30,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1387,1477,23,"г Краснодар, улица им. Героя Яцкова И.В., лите...","Микрорайон ""Губернский""",19.0,20.0,474,2025-06-30,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data.shape

(10612, 108)

Cсылку на фотографии возьмём из колонки photoRenderDTO -> objRenderPhotoUrl

In [4]:
urls_photo = data[['id', 'photoRenderDTO']]
urls_photo.loc[:, 'num_photos'] = urls_photo['photoRenderDTO'].apply(lambda x: len(x))
urls_photo.num_photos.value_counts()

C:\Users\MaxVladislava\AppData\Local\Temp\ipykernel_16868\530275858.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  urls_photo.loc[:, 'num_photos'] = urls_photo['photoRenderDTO'].apply(lambda x: len(x))


1     6683
2     1272
3      856
4      560
5      359
6      228
7      179
8      118
9       87
15      69
10      52
11      40
12      33
0       25
14      25
13      24
16       1
17       1
Name: num_photos, dtype: int64

Как видим, у некоторых объектов может быть несколько фотографий. Для начала создадим словарь, где ключ - objId, а значение - список ссылок на фото.

In [5]:
obj_photos = {}

for index, row in urls_photo.iterrows():
    obj_id = row['id']
    list_urls = [x.get('objRenderPhotoUrl') for x in row['photoRenderDTO']]
    obj_photos[obj_id] = list_urls

Предполагаем, что на вход скрипту данные будут подаваться в виде словаря: ключ - id объекта, а значение - список ссылок на его фото. Скрипт для каждого объекта будет создавать папку с одноимённым названием, затем сохранять в неё его фотографии.

In [6]:
import random
import os
from PIL import Image

In [7]:
def get_photos_from_url(obj_with_urls: dict):
    # создадим общую директорию для фото объектов
    os.makedirs('Objects_photo', exist_ok=True)

    for obj_id, urls in obj_with_urls.items():
        os.makedirs(f'Objects_photo/{obj_id}', exist_ok=True)
        for num, img_url in enumerate(urls):
            img = Image.open(requests.get(img_url, stream=True).raw)
            img.save(f'Objects_photo/{obj_id}/image_{obj_id}_{num}.png')

In [8]:
# проверим на 5 случайных объектах
rand_object_id = random.sample(urls_photo.id.tolist(), k=5)
rand_object = {obj_id: urls for obj_id,urls in obj_photos.items() if obj_id in rand_object_id}
rand_object

{44436: ['https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/9e2beccc-119c-48a6-b1d8-6aa4cb1633ef'],
 46144: ['https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/2791f52d-f1d2-4ba7-9448-4fc9b94f0d7d'],
 46619: ['https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/92ddb70e-a92f-427d-9fee-ada2a8af3ce2',
  'https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/bc48d60a-c220-4f93-8319-186a8f845027',
  'https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/e2b7a3f4-12b3-4732-b1c6-bf794a92d552',
  'https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/9f156dce-375e-45ff-af71-1a3d17e78881'],
 46983: ['https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/f41cfa98-6858-45de-8616-3ad3aa672025',
  'https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/eda7695a-7446-41da-a249-7627298aa853',
  'https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/fae7825e-8ce5-4bd0-90c4-b92383baffd9',
  'https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/c2458215-8155-4157-90a4-929948e1cbc0',
  'https://xn--80az8a.xn--d1aqf.xn--p1a

In [9]:
get_photos_from_url(rand_object)